<a href="https://colab.research.google.com/github/kuzmix/GCPL_RNN/blob/Colab-training/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""For google colab workflow - mounts Google Drive and go to it"""
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('./gdrive/MyDrive/Projects/GCPL_RNN')

In [ ]:
"""For local machine - go to root directory"""
import os
os.chdir('../')

In [ ]:
"""Imports all necessary libs"""
import os
import sys
import numpy as np
from sys import getsizeof
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from itertools import product
import pickle
import math
import shutil
import torch
from torch.utils.data import Dataset
import scipy as sp
from scipy.optimize import curve_fit
import sklearn
from Code.setup import *
import datetime as dt
import time
import torch.nn as nn
import copy
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
import RNN_for_GCPL.setup

In [ ]:
"""For compatibility - cd to folder with data and models"""
os.chdir('../RNN_for_GCPL/')

In [ ]:
savepath = r'data/v4/30/main'
dataset_validate = GCPL_dataset_resampled3(r'data/v4/120/validate')
dataset_rs2 = GCPL_dataset_resampled3(savepath)

In [ ]:
seedEverything(seed=DEFAULT_RANDOM_SEED)
input_dim = 2
output_dim = 20
num_layers = 2
bidir= True
lr = 5e-4
init_model = MyGRU(input_dim, output_dim, num_layers=num_layers,bidir = bidir)
criterion = nn.MSELoss(reduction='none')
best_model = MyGRU(input_dim, output_dim, num_layers=num_layers,bidir = bidir)
# kf = KFold(5, shuffle=True, random_state=DEFAULT_RANDOM_SEED)
gkf = GroupKFold(4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
models = []
# params = [ (30,3,False), (60, 4, True)]
params = [(20, 2, False), (20, 2, True), (30, 3, True)]
for a,b,c in params:
    model = MyGRU(input_dim, a, num_layers=b, bidir=c)
    models.append(model)
sampling = [30, 60, 120, 180, 300]

In [ ]:
probs = balancing(dataset_rs2, 10)
soh, info = statistics(dataset_rs2)
clear_output()

In [ ]:
class BucketSampler(torch.utils.data.Sampler):
    
    def __init__(self, lengths, buckets=(50,500,50), shuffle=True, batch_size=32, drop_last=False):
        
        super().__init__(lengths)
        
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.drop_last = drop_last
        
        assert isinstance(buckets, tuple)
        bmin, bmax, bstep = buckets
        assert (bmax - bmin) % bstep == 0
        
        buckets = defaultdict(list)
        for i, length in enumerate(lengths):
            if length > bmin:
                bucket_size = min((length // bstep) * bstep, bmax)
                buckets[bucket_size].append(i)
                
        self.buckets = dict()
        for bucket_size, bucket in buckets.items():
            if len(bucket) > 0:
                self.buckets[bucket_size] = torch.tensor(bucket, dtype=torch.int, device='cpu')
        
        # call __iter__() to store self.length
        self.__iter__()
            
    def __iter__(self):
        
        if self.shuffle == True:
            for bucket_size in self.buckets.keys():
                self.buckets[bucket_size] = self.buckets[bucket_size][torch.randperm(self.buckets[bucket_size].nelement())]
                
        batches = []
        for bucket in self.buckets.values():
            curr_bucket = torch.split(bucket, self.batch_size)
            if len(curr_bucket) > 1 and self.drop_last == True:
                if len(curr_bucket[-1]) < len(curr_bucket[-2]):
                    curr_bucket = curr_bucket[:-1]
            batches += curr_bucket
            
        self.length = len(batches)
        
        if self.shuffle == True:
            random.shuffle(batches)
            
        return iter(batches)
    
    def __len__(self):
        return self.length


In [ ]:
def collate_batch_length(batch):
    sample_list = []
    label_list = []
    lengths = []
    for i in batch:
        sample = np.stack([i['E'], i['I']],axis=-1)
        sample_list.append(torch.tensor(sample, dtype=torch.float32))
        label_list.append(i['SoH'])
        lengths.append(len(i['E']))
    sequence_pad = nn.utils.rnn.pad_sequence(sample_list)
    labels = torch.tensor(label_list, dtype=torch.float32)
    lengths_torch = torch.tensor(lengths, dtype=torch.float32)
    return sequence_pad, labels, lengths_torch  


In [ ]:
missed_length = []
n_bins = 10
_ , ind = statistics(dataset_rs2)
for k, (train_indices, val_indices) in enumerate(gkf.split(dataset_rs2, groups=info.Pouch)):
    seedEverything(seed=DEFAULT_RANDOM_SEED)
    train_set = torch.utils.data.Subset(dataset_rs2, train_indices)
    
    min_length =  ind.loc[train_indices,'Len'].min()
    max_length = ind.loc[train_indices,'Len'].max()
    max_length += n_bins - (max_length-min_length)% n_bins
    print(min_length, max_length)
    val_set = torch.utils.data.Subset(dataset_rs2, val_indices)
    sampler = torch.utils.data.WeightedRandomSampler(probs[train_indices], len(train_indices)) 
    bucket_sampler = BucketSampler(ind.loc[train_indices,'Len'].to_numpy(), buckets = (min_length, max_length, n_bins), batch_size=batch_size)
    # train_loader = torch.utils.data.DataLoader(train_set, batch_size =1,  batch_sampler=bucket_sampler, collate_fn=collate_batch_length)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size =batch_size, shuffle=True, collate_fn=collate_batch_length)

    val_loader = torch.utils.data.DataLoader(val_set, batch_size, shuffle=True, collate_fn=collate_batch)
    miss_ = []
    for data, labels, length in train_loader:
        m = data.shape[0]- length.mean()
        miss_.append([m, data.shape[0],length.mean()])
    miss_ = torch.tensor(miss_)
    missed_length.append(miss_)
    print(miss_.mean(axis=0))


In [ ]:
draw_every = 20
num_epochs = 100
save_every = 10
batch_size = 64
n_bins = 10
drop_after = 20
model_version = 14
for m in sampling[3:]:
    dataset_rs2 = GCPL_dataset_resampled3(f'data/v4/{m}/main')
    dataset_validate = GCPL_dataset_resampled3(f'data/v4/{m}/validate')
    for l, init_model in enumerate(models):
        path = f'./models/v{model_version}/{m}/{l}'
        for k, (train_indices, val_indices) in enumerate(gkf.split(dataset_rs2, groups=info.Pouch)):
            seedEverything(seed=DEFAULT_RANDOM_SEED)
            min_length =  info.loc[train_indices,'Len'].min()
            max_length = info.loc[train_indices,'Len'].max()
            max_length += n_bins - (max_length-min_length)% n_bins
            prob_train = probs.copy()
            prob_train[val_indices] = 0
            
            val_set = torch.utils.data.Subset(dataset_rs2, val_indices)
            # sampler = torch.utils.data.WeightedRandomSampler(probs[train_indices], len(train_indices)) 
            # train_loader = torch.utils.data.DataLoader(train_set, batch_size, collate_fn=collate_batch, sampler=sampler)
            # val_loader = torch.utils.data.DataLoader(val_set, batch_size, shuffle=True, collate_fn=collate_batch)
            
            bucket_sampler_val = BucketSampler(info.loc[val_indices,'Len'].to_numpy(), buckets = (min_length, max_length, n_bins), batch_size=batch_size)
            
            val_loader = torch.utils.data.DataLoader(val_set, batch_size =1,  batch_sampler=bucket_sampler_val, collate_fn=collate_batch)
            model = copy.deepcopy(init_model)
            model.to(device)
            optimizer =  torch.optim.Adam(model.parameters(), lr=lr)
            sheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=6)
            handler = ModelHandler(model, 1e6, f'20 neurons, 2 layers, 120s sampling, 80% train, 20% val, k-fold No {k}.', path= path, kfold=True)
            pp = ProgressPlotter()
            counter = 0
            for i in range(num_epochs):
                balance = torch.multinomial(torch.tensor(prob_train) , len(train_indices), replacement=True)
                train_set = torch.utils.data.Subset(dataset_rs2, balance)
                bucket_sampler_train = BucketSampler(info.loc[balance.numpy(),'Len'].to_numpy(), buckets = (min_length, max_length, n_bins), batch_size=batch_size)
                train_loader = torch.utils.data.DataLoader(train_set, batch_size =1,  batch_sampler=bucket_sampler_train, collate_fn=collate_batch)

                model.train()
                loss_val = 0
                loss_train = 0
                for data, labels in train_loader:
                    optimizer.zero_grad()
                    preds = model(data.to(device))
                    loss = torch.sum(criterion(preds, labels.to(device)))
                    loss.backward()
                    optimizer.step()
                    loss_train += loss.detach().cpu()
                
                
                with torch.no_grad():
                    loss_train = loss_train/len(train_set)
                    pp.add_scalar('loss_train', loss_train.detach().cpu().numpy())
                    model.eval()
                    for data, labels in val_loader:
                        preds = model(data.to(device))
                        loss = torch.sum(criterion(preds, labels.to(device)))
                        loss_val += loss.detach().cpu()
                    loss_val = loss_val/len(val_set)
                    sheduler.step(loss_val)
                    handler.check_loss(loss_val, draw_every)
                    pp.add_scalar('loss_val', loss_val.detach().cpu().numpy())
                    if loss_val > handler.best_loss:
                        counter +=1
                    else:
                        counter = 0
                    if (i+1)%draw_every == 0:
                        pp.display([['loss_train', 'loss_val']])
                    print(i, counter, loss_train*2500, loss_val*2500)
                    if counter > drop_after:
                        break
                if (i+1)%save_every==0:
                    handler.save(kfold_number=k)
                    
                    
            handler.add_pp(pp)
            handler.display()
            handler.save(kfold_number=k)
            with torch.no_grad():
                loss_val = []
                for data, labels in val_loader:
                    preds = model(data.to(device))
                    loss = criterion(preds, labels.to(device))
                    loss_val.append(loss)
                loss_val = torch.hstack(loss_val)
                print(loss_val.mean(), loss_val.std())


In [ ]:
"""Loss checking"""
loss = []
model = copy.deepcopy(init_model) 
path= f'./models/v127/2'
handler = ModelHandler(model, 1e6, path= f'./models/v2/', kfold=True)
batches = next(os.walk(path))[1]
loss_ = []
epochs_ = []
for batch in batches:
    batchpath = path+ '/' + batch
    handler.load(batchpath)
    loss_.append(handler.best_loss.cpu())
loss.append(loss_)
loss = np.array(loss)*2500
print(str(np.mean(loss))+ '±' + str(np.std(loss)))
loss

In [ ]:
def load(path_):
    if path_.split('.')[-1] == 'pt':
        path = '/'.join(path.split('/')[:-1])
        name = path_.split('/')[-1]
        checkpoint = torch.load(path+ '/' + name, map_location=torch.device('cpu'))
        loss = [checkpoint['loss'].numpy()]
    else:
        path = path_
        files = next(os.walk(path_))[2]
        x = lambda x: int(x.split('_')[-1].split('.')[0].split('(')[0])
        files.sort(key=x)
        loss = []
        for name in files:
            checkpoint = torch.load(path+ '/' + name, map_location=torch.device('cpu'))
            loss.append(checkpoint['loss'].numpy())
    return np.array(loss)


In [ ]:
path = f'./models/v10'
loss = {}
for root, dirs, files in os.walk(path):
    if dirs:
        continue
    if not files:
        continue

    loss[root] = load(root)*2500
loss

In [ ]:
def collate_batch_named(batch, named=True):
    sample_list = []
    label_list = []
    pouch = []
    cycle = []
    for i in batch:
        sample = np.stack([i['E'], i['I']],axis=-1)
        sample_list.append(torch.tensor(sample, dtype=torch.float32))
        label_list.append(i['SoH'])
        pouch.append(i['Pouch'])
        cycle.append(i['Cycle'])
    sequence_pad = nn.utils.rnn.pad_sequence(sample_list)
    labels = torch.tensor(label_list, dtype=torch.float32)
    if not named:
        return sequence_pad, labels    
    else:
        return sequence_pad, labels, pouch, cycle

In [ ]:
def detailed_loss(handler, val_loader):
    with torch.no_grad():
        loss = []
        pouches = []
        soh = []
        pred = []
        cycles = []
        for data, labels, pouch, cycle in val_loader:
            preds = handler.best_model(data)
            pred.extend(preds)
            loss_ = criterion(preds, labels)
            loss.append(loss_)
            pouches.extend(pouch)
            cycles.extend(cycle)
            soh.extend(labels)
        
        loss = torch.hstack(loss)
        soh = torch.Tensor(soh)*50+50
        pred = torch.Tensor(pred)*50+50
        loss *= 2500 
        results = pd.DataFrame({'Pouch':pouches,'Cycle':cycles, 'Loss':loss, 'SoH':soh,'Pred':pred} )
        # results.sort_values(by='Loss',axis=0, ascending=0,inplace=True)
        return results

In [ ]:
batch_size = 64
model_version = 10
losses = []
for k in range(3):
    for s in sampling:
        datapath = f'data/v4/{s}/'
        modelpath = f'./models/v{model_version}/{s}/{k}'
        dataset_validate = GCPL_dataset_resampled3(datapath+'validate')
        dataset_rs2 = GCPL_dataset_resampled3(datapath+'main')
        main_loader = torch.utils.data.DataLoader(dataset_rs2, batch_size, shuffle=False, collate_fn=collate_batch_named)
        val_loader = torch.utils.data.DataLoader(dataset_validate, batch_size, shuffle=False, collate_fn=collate_batch_named)
        for i in next(os.walk(modelpath))[1]:
            init_model = copy.deepcopy(models[k])
            path = modelpath + f'/{i}'
            print(path)
            handler = ModelHandler(init_model)
            handler.load(path)
            result_main = detailed_loss(handler, main_loader)
            result_val = detailed_loss(handler, val_loader)
            result = pd.concat([result_main, result_val])
            loss = pd.concat([result.groupby('Pouch').Loss.mean(), result.groupby('Pouch').Loss.std()], axis=1)
            loss.columns = [f'{s}/{k}/{i}_Mean', f'{s}/{k}/{i}_Std']
            losses.append(loss)
f_loss = pd.concat(losses, axis=1)


In [ ]:
loss = {}
for s in sampling:
    l_ = {}
    for i in range(3):
        path = f'./models/v10/{s}/{i}'
        batches = next(os.walk(path))[1]
        loss_ = []
        for batch in batches:
            batchpath = path+ '/' + batch
            a = load(batchpath)
            loss_.append(a)
        loss_ = torch.tensor(loss_)*2500
        l_[i] = loss_
        print(s, i, f'{loss_.mean():.3}±{loss_.std():.3}')
    loss[s] = l_

In [ ]:
length = []
soh = []
info = pd.DataFrame({
            'Pouch':[],
            'SoH':[],
            "Len":[],
            })
for i in range(len(dataset_rs2)):
    one = len(dataset_rs2[i]['E'])
    soh = dataset_rs2[i]['SoH']*50+50
    if np.any(np.isnan(dataset_rs2[i]['E'])):
        print(i)
    pouch = dataset_rs2[i]['Pouch']
    if i%1000 == 0:
        print(i)
    info.loc[len(info)] = [pouch, soh, one]
info